# Transfer Learning is for the Birds

Patrick Wagstrom &lt;patrick@wagstrom.net&gt;

This notebook provides an overview of the end-to-end process for applying transfer learning to retrain an object detection and classification model and then export that model to a tflite model that can be used on a Google Edge TPU. This is loosely based on https://github.com/dctian/DeepPiCar/blob/master/models/object_detection/code/tensorflow_traffic_sign_detection.ipynb and https://coral.ai/docs/edgetpu/retrain-detection/.

# Configuration Settings

Google Colab makes it really easy to create forms to control experiments. It's generally a best practice to put anything that might need to be customized for a model up front in a form. This is a little like the block of constants you might see at the top of a normal python script.

In [0]:
#@title Global Training Settings

# Number of training steps.
num_steps =   1000# @param {type:"integer"}
# Number of evaluation steps.
num_eval_steps = 50 # @param {type:"integer"}

#@markdown ---
selected_model = 'ssd_mobilenet_v2_quantized' # @param {type: "string"}

training_folder = '/content/drive/My Drive/Data/Bird Classifier/' # @param {type: "string"}

training_file = '/content/drive/My Drive/Data/Bird Classifier/data.csv' # @param {type: "string"}

image_prefix_orig = '/Users/pwagstro/Google Drive' # @param {type: "string"}

image_prefix_new = '/content/drive/My Drive' # @param {type: "string"}
working_dir = '/content/work' #@param {type:"string"}

quantized_output_file = "bird_classifier_quantized.tflite" #@param {type:"string"}
float_output_file = "bird_classifier_float.tflite" #@param {type: "string"}

persistence_dir = '/content/drive/My Drive/Models/Bird Classifier ' #@param {type: "string"}
#@markdown ---
#@markdown Size for Input Images
scale_image_width = 800 #@param {type:"integer"}
scale_image_height = 600 #@param {type: "integer"}


# Specify Tensorflow Version

Unfortunately, Google doesn't do a great job with keeping their scripts working across tensorflow versions. This is a bit of Google Colab magic that specifes the version of Tensorflow that we will use.

In [0]:
%tensorflow_version 1.x

# Get Machine Information

Verify that we're running on the right kind of machine. These scripts will use an nvidia GPU, but not a TPU.

In [0]:
import subprocess
gpu_info = subprocess.run('nvidia-smi', stdout=subprocess.PIPE).stdout.decode('utf-8')
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Mar  4 18:19:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Install Dependencies for EdgeTPU Compilation

Later on in the notebook there are steps that will compile the model down to a quantized TFLite model. By default, the Google Colab Machines do not have the utilites for edgetpu compilation installed. Fortunately, we can get around that pretty easily by installing it from the Google provided Debian repositories.

In [0]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
! sudo apt-get update
! sudo apt-get install edgetpu

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  16743      0 --:--:-- --:--:-- --:--:-- 17184
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu18

In [0]:
import numpy as np
import pandas as pd
import re
from PIL import Image
from tqdm.autonotebook import tqdm

import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
# IPython Display Hack for Dark Mode
#
# This is really needed for when you're using TQDM with Google Colab's Dark Mode
# It preprends a little bit of CSS to make the progress bars look better
from IPython.display import Math, HTML, display

def set_css_in_cell_output():
  display(HTML("""<style>label.widget-label { color: #ffffff !important; }
div.p-Widget { color: #ffffff; }
</style>"""))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)

In [0]:
!mkdir -p {working_dir}
%cd /content
!rm -rf /content/models
!git clone --quiet https://github.com/tensorflow/models.git

# !apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

# !pip install -q Cython contextlib2 pillow lxml matplotlib

# !pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.


# copy additional scripts necessary for execution
!cp /content/drive/My\ Drive/Code/Tensorflow/generate_tfrecord.py {working_dir}

import os
os.environ['PYTHONPATH'] += f':/content/models/research/:/content/models/research/slim/:{working_dir}'


/content
/content/models/research


In [0]:
# run the self tests on the model building code
!python object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

In [0]:
MODELS_CONFIG = {
    'ssd_mobilenet_v2_quantized': {
        'model_name': 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03',
        'pipeline_file': 'ssd_mobilenet_v2_quantized_300x300_coco.config',
        'batch_size': 12
    }
}

In [0]:
# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Labeling the Training Data

This probably the most annoying part of the project - labeling the images from training data. There are a number of different tools that we can use to manage this, in the past I've used [labelimg](https://github.com/tzutalin/labelImg) which is a Python program that uses the QT GUI toolkit to manage image labeling. This time I'm trying to use [VGG Image Annotator (VIA)](https://www.robots.ox.ac.uk/~vgg/software/via/) to label my data. Although the interface for VIA isn't as good - it provides a nice stepping stone to using it for more precise image labeling beyond bounding boxes.

## Training Data

When all is said and done, you should have a training file that looks like this. As you can see, we're only doing coarse bounding box object detection in this model. In the future we may switch to doing higher quality object segmentation, but that requires a lot more work on tracing out the birds, and I'm kinda lazy.

```
filename,width,height,class,xmin,ymin,xmax,ymax
2019-04-16-095558.jpg,640,480,Speed Limit 25,78,111,153,204
2019-04-16-095558.jpg,640,480,Green Traffic Light,223,132,251,184
2019-04-16-095558.jpg,640,480,Stop Sign,298,129,361,193
2019-04-16-095558.jpg,640,480,Red Traffic Light,393,129,417,182
```

In [0]:
df = pd.read_csv(training_file)
df['filename'] = [re.sub(image_prefix_orig,image_prefix_new, str(x)) for x in df['filename']]
df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,/content/drive/My Drive/Data/Bird Classifier/I...,4032,3024,sparrow,696,1648,833,1779
1,/content/drive/My Drive/Data/Bird Classifier/I...,4032,3024,sparrow,2029,1621,2099,1700
2,/content/drive/My Drive/Data/Bird Classifier/I...,4032,3024,sparrow,1971,1688,2044,1761
3,/content/drive/My Drive/Data/Bird Classifier/I...,4032,3024,sparrow,1849,1703,1919,1758
4,/content/drive/My Drive/Data/Bird Classifier/I...,4032,3024,sparrow,1934,1706,1977,1776


In [0]:
image_dir = os.path.join(working_dir, 'images')
os.makedirs(image_dir, exist_ok=True)

def scale_image(fn: str):
    image = Image.open(fn)
    original_size = image.size

    output_fname = os.path.join(image_dir, os.path.split(fn)[1])

    output_size = ()
    if os.path.isfile(output_fname):
        image = Image.open(output_fname)
        output_size = image.size    
    else:
        # use Image.thumbnail rather than Image.resize because we want to preserve aspect ratio
        image.thumbnail((scale_image_width, scale_image_height), resample=Image.ANTIALIAS)
        output_size = image.size
        image.save(output_fname)

    scale_ratio = float(output_size[0]) / float(original_size[0])

    return { "filename": fn, "new_filename": output_fname, "width": output_size[0],
             "height": output_size[1], "scale": scale_ratio }

image_scaling_map = {}
for fn in tqdm(set(df['filename'])):
    image_scaling_map[fn] = scale_image(fn)

df['scale'] = [image_scaling_map[fn]['scale'] for fn in df['filename']]
for k in ['width', 'height', 'xmin', 'ymin', 'xmax', 'ymax']:
    df[k] = df[k] * df['scale']
    df[k] = df[k].astype(int)

df['filename'] = [image_scaling_map[fn]['new_filename'] for fn in df['filename']]
df.head() 

,filename,width,height,class,xmin,ymin,xmax,ymax,scale
0,/content/work/images/IMG_0351.jpg,800,600,sparrow,138,326,165,352,0.198413
1,/content/work/images/IMG_0351.jpg,800,600,sparrow,402,321,416,337,0.198413
2,/content/work/images/IMG_0351.jpg,800,600,sparrow,391,334,405,349,0.198413
3,/content/work/images/IMG_0351.jpg,800,600,sparrow,366,337,380,348,0.198413
4,/content/work/images/IMG_0351.jpg,800,600,sparrow,383,338,392,352,0.198413


In [0]:
np.random.seed(0)

# Generate a per-file mask
mapping = {x:np.random.rand() < 0.8 for x in set(df['filename'])}
df['testtrain'] = [mapping[x] for x in df['filename']]

# save the testing and training data
df_train = df[df['testtrain'] == True]
df_test = df[df['testtrain'] == False]

del df_train['testtrain']
del df_test['testtrain']

df_train.to_csv(os.path.join(working_dir, 'train_labels.csv'), index=False)
df_test.to_csv(os.path.join(working_dir, 'test_labels.csv'), index=False)

# Explore the Data and Annotate Some Images

> Indented block



# Create the TFRecord Files

First, we need to create the `.pbtxt` file that maps labels. Of course, this isn't a real JSON format, because of course it's not.

The format consists of repeated records that look a little like unquoted JSON with and `id` and a `name` field. Indexing should start with `1` because `0` is the background label. Here's an example record:

```
item {
    id: 1
    name: 'american_goldfinch'
}
```

We don't need to put the records in alphabetical order, I do that just because it's a little nicer for me.

In [0]:
with open(os.path.join(working_dir, "label_map.pbtxt"), "w") as f:
    for idx, val in enumerate(sorted(list(set(df['class'])))):
        f.write('item {\n    id: ' + str(idx+1) + '\n    name: \'' + val + '\'\n}\n\n')

The actual `tfrecord` files are a compact format for feeding all of the data into the model. Personally, I think this is overkill for sample models as by the time you get to the scale that you really need this compact of a format you're likely working at a super large scale anyway and these methods won't work for you anyway.

In [0]:
%cd {working_dir}

# Generate `train.record`
!python generate_tfrecord.py --csv_input={working_dir}/train_labels.csv \
  --output_path={working_dir}/train.record --img_path=data/images/train \
  --label_map {working_dir}/label_map.pbtxt

# Generate `test.record`
!python generate_tfrecord.py --csv_input={working_dir}/test_labels.csv \
  --output_path={working_dir}/test.record --img_path=data/images/test \
  --label_map {working_dir}/label_map.pbtxt


/content/work

W0304 18:24:52.547324 139965369542528 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/work/train.record

W0304 18:24:56.585836 139924377474944 module_wrapper.py:139] From /content/models/research/object_detection/utils/label_map_util.py:138: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/work/test.record


In [0]:
test_record_fname = os.path.join(working_dir, 'test.record')
train_record_fname = os.path.join(working_dir, 'train.record')
label_map_pbtxt_fname = os.path.join(working_dir, 'label_map.pbtxt')

## Download and unpack the selected model

This connects to the TensorFlow Model Zoo and downloads and unpacks the selected model. It might not be necessary if we've already managed to go and grab the model as part of checking out the model zoo earlier in the setup.

In [0]:
%mkdir -p {working_dir}
%cd {working_dir}

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL_FILE = MODELS_CONFIG[selected_model]['model_name'] + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = f'{working_dir}/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
print(f'Fine Tuning Checkpoint: {fine_tune_checkpoint}')

/content/work
Fine Tuning Checkpoint: /content/work/pretrained_model/model.ckpt


# Create the training pipeline

Because we're using a pre-trained model from Google, we can also use their existing training pipeline to help us in this step. However, there are a number of things that we need to tweak to make it work properly.

First, check to see if the file is present. If not, error out.

In [0]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), f'`{pipeline_fname}` does not exist'

Next, do a bunch of tweaks of of the pipeline according to the settings we have and the files we've created. In many of the original demos this has another step with a function called `get_num_classes` that gets the classes from your `label_map.pbtxt` file. But, becuase that file has been created earlier in the process, we don't need to do that here and can just give it the numebr of labels.

In [0]:
import re

num_classes = len(set(df['class']))

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

# Show the Pipeline

Let's take a quick look at the modified pipeline to see what we're using to retrain the model

In [0]:
!cat {pipeline_fname}

# Quantized trained SSD with Mobilenet v2 on MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 12
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        asp

In [0]:
model_dir = 'training/'
# remove the content in the model training directory to give a clean start
if (os.path.exists(model_dir)):
    shutil.rmtree(model_dir)
os.makedirs(model_dir, exist_ok=True)

# Train the Actual Model

The way that most people train these models is by shelling out to a python program to train the model. Overall that's okay, but it's not a great way to think about the problem because it hides a lot of what is going on behind the scenes. I'd rather this be a python module that is easier to import so we can easily tweak the code.

In [0]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0304 18:25:12.192824 139969822328704 module_wrapper.py:139] From /content/models/research/object_detection/utils/config_util.py:102: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0304 18:25:12.195815 139969822328704 module_wrapper.py:139] From /content/models/research/object_detection/model_lib.py:628: The name tf.logging.warning is deprecated. Please use tf.compat.v1.logging.warning instead.

W0304 18:25:12.195952 139969822328704 model_lib.py:629] Forced number of epochs for all eval validations to be 1.

W0304 18:25:12.196066 139969822328704 module_wrapper.py:139] From /content/models/resea

# Export a Trained Inference Graph

In [0]:
output_directory = f'{working_dir}/fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

training/model.ckpt-1000
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0304 18:34:18.872560 140037370951552 module_wrapper.py:139] From /content/models/research/object_detection/export_inference_graph.py:145: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0304 18:34:18.878009 140037370951552 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:402: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0304 18:34:18.878276 140037370951552 module_wrapper.py:139] From /content/models/research/object_detection/exporter.py:121: The name tf.placeholder is deprecated. Please use tf.com

In [0]:
!ls {output_directory}

checkpoint			model.ckpt.index  saved_model
frozen_inference_graph.pb	model.ckpt.meta
model.ckpt.data-00000-of-00001	pipeline.config


In [0]:
# https://medium.com/tensorflow/training-and-serving-a-realtime-mobile-object-detector-in-30-minutes-with-cloud-tpus-b78971cf1193
# create the tensorflow lite graph
!python /content/models/research/object_detection/export_tflite_ssd_graph.py \
    --pipeline_config_path={pipeline_fname} \
    --trained_checkpoint_prefix='{last_model_path}' \
    --output_directory='{output_directory}' \
    --add_postprocessing_op=true

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



W0304 18:34:42.386855 140685558839168 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph.py:133: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0304 18:34:42.391050 140685558839168 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph_lib.py:193: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.


W0304 18:34:42.391336 140685558839168 module_wrapper.py:139] From /content/models/research/object_detection/export_tflite_ssd_graph_lib.py:237: The name tf.placeholder is deprecated. Ple

# Convert to a tflite model

In [0]:
!echo "CONVERTING frozen graph to quantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/{quantized_output_file}' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

CONVERTING frozen graph to quantized TF Lite file...
2020-03-04 18:35:00.887677: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-04 18:35:00.900969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-04 18:35:00.901520: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-03-04 18:35:00.901799: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-03-04 18:35:00.903438: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-03-04 18:35:00.905086: I tensorflow/stream_executor/platform/default/dso_loa

In [0]:
!echo "CONVERTING frozen graph to unquantized TF Lite file..."
!tflite_convert \
  --output_file='{output_directory}/{float_output_file}' \
  --graph_def_file='{output_directory}/tflite_graph.pb' \
  --input_arrays='normalized_input_image_tensor' \
  --output_arrays='TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3' \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,300,300,3 \
  --change_concat_input_ranges=false \
  --allow_nudging_weights_to_use_fast_gemm_kernel=true \
  --allow_custom_ops

CONVERTING frozen graph to unquantized TF Lite file...
2020-03-04 18:35:10.464016: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-03-04 18:35:10.476859: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-03-04 18:35:10.477396: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla P100-PCIE-16GB major: 6 minor: 0 memoryClockRate(GHz): 1.3285
pciBusID: 0000:00:04.0
2020-03-04 18:35:10.477650: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-03-04 18:35:10.479203: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2020-03-04 18:35:10.480788: I tensorflow/stream_executor/platform/default/dso_l

# Compile to EdgeTPU

In [0]:
%cd {output_directory}
! edgetpu_compiler -s {output_directory}/{quantized_output_file}

/content/work/fine_tuned_model
Edge TPU Compiler version 2.0.291256449

Model compiled successfully in 514 ms.

Input model: /content/work/fine_tuned_model/bird_classifier_quantized.tflite
Input size: 4.67MiB
Output model: bird_classifier_quantized_edgetpu.tflite
Output size: 5.40MiB
On-chip memory available for caching model parameters: 7.62MiB
On-chip memory used for caching model parameters: 5.20MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 99
Operation log: bird_classifier_quantized_edgetpu.log

Model successfully compiled but not all operations are supported by the Edge TPU. A percentage of the model will instead run on the CPU, which is slower. If possible, consider updating your model to use only operations supported by the Edge TPU. For details, visit g.co/coral/model-reqs.
Number of operations that will run on Edge TPU: 98
Number of operations that will run on CPU: 1

Operator                

# Pesist in Google Drive


In [0]:
! mkdir -p "{persistence_dir}"
! cp {output_directory}/*.tflite "{persistence_dir}"

# Test the Inference in Google Colab

Now that we've generated a model, we can test it using some of the data in our test set that we witheld earlier during the training process.

For the most part, this is boilerplate code that is found in most Tensorflow projects that use object detection.

In [0]:
test_image_paths = set(df_test['filename'])
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")

In [0]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(pb_fname, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(label_map_pbtxt_fname)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
IMAGE_SIZE = (16, 12)


def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

/content/models/research/object_detection


In [0]:
# running inferences.  This should show images with bounding boxes
%matplotlib inline

print(f'Running inferences on {test_image_paths}')
for image_path in tqdm(test_image_paths):
    image = Image.open(image_path)
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=2)
    plt.figure(figsize=IMAGE_SIZE)
    plt.imshow(image_np)